In [2]:
import numpy as np
import os
import sys
import json
import pickle
import re
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

### Read Stopwords

In [3]:
stopwords = []
with open('../../data/meta/stopwords','r') as f:
    for line in f:
        stopwords.append(line.rstrip())

### Find Files

In [4]:
myfiles = []
classes = []
for root, dirs, files in os.walk("../../data/cancer_corpi/articles/", topdown=False):
    for name in files:
        if '.txt' in name:
            classes.append(name.split('/')[-1][:-4])
            myfiles.append(os.path.join(root, name))

### Read Files

In [5]:
records = []
labels = []
for m,myfile in enumerate(myfiles):
    with open(myfile,'r') as f:
        block = []
        prev_blocks = []
        for line in f:
            line = line.rstrip()
            if (len(line) == 0):
                prev_blocks.append(block)
                if (len(prev_blocks) > 5):
                    prev_blocks = prev_blocks[1:]
                if (len(block) > 5):
                    for p,prev_block in enumerate(prev_blocks):
                        for prev_line in prev_block:
                            if 'Author information:' == prev_line:
                                title = ' '.join(prev_blocks[p-2])
                                authors = ' '.join(prev_blocks[p-1])
                                abstract = ' '.join(block)
                                records.append((title,authors,abstract))
                                labels.append(classes[m])
                block = []
            else:
                block.append(line)

### Clean

In [8]:
records_clean = []
count = 0
for i in tqdm(range(len(records))):

    label = labels[i]
    title,authors,abstract = records[i]
    if '[Article' in title:
        pass
    else:
        
        #Title
        title = title.replace("'s","")
        title = title.replace("'","")
        title = re.sub(r'[^a-zA-Z]',' ', title)
        title = title.lower()
        words = title.split(' ')
        words = [word for word in words if len(word) > 0]
        words = [word for word in words if word not in stopwords]
        title = ' '.join(words)

        #Abatract
        abstract = abstract.replace("'s","")
        abstract = abstract.replace("leukaemia","leukemia")
        if 'leukaemia' in abstract:
            count = count + 1
        abstract = abstract.replace("'","")
        abstract = re.sub(r'[^a-zA-Z]',' ', abstract)
        abstract = abstract.lower()
        words = abstract.split(' ')
        words = [word for word in words if len(word) > 0]
        words = [word for word in words if word not in stopwords]
        abstract = ' '.join(words)

        #Auhor
        authors = re.sub(r'\([^)]*\)', '', authors)
        authors = authors.replace('.','')
        authors = authors.lower()
        authors = authors.split(',')

        #Store
        records_clean.append((title,abstract,authors,label))
print(count)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39787/39787 [00:13<00:00, 2908.77it/s]

0


### Store

In [35]:
data = {}
data['records'] = records_clean
pickle.dump( data, open( "../../data/cancer_corpi/articles/pickle/processed.p", "wb"))